#**Using GridSearchCV in KNNWithMeans, KNNBasic, KNNBaseline**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')

In [ ]:
movies=pd.read_csv('/content/drive/My Drive/Dataset/movies.csv')
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings=pd.read_csv('/content/drive/My Drive/Dataset/Ratings .csv')
ratings

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
userdata=pd.read_csv('/content/drive/My Drive/Dataset/user_data.csv')
userdata

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,1060


In [ ]:
df=ratings.merge(userdata, on='UserID',how='left').merge(movies, on='MovieID',how='left')
df

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df=df.drop(columns='Timestamp')
df

,UserID,MovieID,Rating,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,M,25,6,11106,Sophie's Choice (1982),Drama


In [ ]:
df.shape

(1000209, 9)

In [ ]:
!pip install surprise

In [ ]:
from surprise import Reader, Dataset , KNNWithMeans

In [ ]:
reader = Reader(rating_scale=(1,5))
#Loads from Dataframe into data
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)

In [ ]:
#Compute similarity between items
methods = {
    'name':'cosine',
    'user_based': False
}
algo = KNNWithMeans(k=10,sim_options=methods)

In [ ]:
trainset = data.build_full_trainset()
#Fit KNNwithmean
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
def prediction(userid,movieid):
  pred = algo.predict(userid,movieid)
  rating =  pred.est
  print('User {} rate {:.1f}/5 of movieid {}'.format(userid,rating,movieid))

In [ ]:
prediction(35,45)

User 35 rate 2.9/5 of movieid 45


In [ ]:
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

 ## **GridSearchCV**

## ***1.***   ***KNNWithMeans***

In [ ]:
#KNNWithMeans
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [ ]:
alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# perform train test split on the best parametres
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNWithMeans(name='msd', min_support=5, user_based=False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9306
MAE:  0.7391


0.739127786331252

In [ ]:
# using cosine similarity
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNWithMeans(name='cosine', min_support=5, user_based=False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9295


0.9295199820631201

In [ ]:
# KNNWithMeans using cosine similarity only
from surprise import KNNWithMeans
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV


reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [5],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.89408056190716
MAE: 0.702771576599453
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}



##***2.   KNNBasic***



In [ ]:
# KNNBasic
from surprise import KNNBasic
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [ ]:
# perform train test split using best parameters
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBasic(name = 'msd', min_support = 5, user_based = False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9262
MAE:  0.7307


0.7306556466223432

In [ ]:
#using cosine similarity
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBasic(name = 'cosine', min_support = 5, user_based = False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9289
MAE:  0.7317


0.7316674140159196

In [ ]:
# KNNBasic using cosine similarity only
from surprise import KNNBasic
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV


reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [5],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.999738826435713
MAE: 0.7804487811659365
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}


##***3. KNNBaseline***

In [ ]:
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [ ]:
# perform train test split on best parameters
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBaseline(name = 'msd', min_support = 5, user_based = False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8952
MAE:  0.7063


0.7062906993749141

#*KNNBaseline* (Check which parameter is best from pearson_baseline and cosine similaritry)

In [ ]:
#KNNBaseline using pearson baseline and cosine similaritry
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["pearson_baseline", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [ ]:
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBaseline(name = 'pearson_baseline', min_support = 5, user_based = False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8965
MAE:  0.7085


0.7084721749790376

In [ ]:
from surprise import KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBaseline(name = 'pearson_baseline', min_support = 3, user_based = False)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8982
MAE:  0.7092


0.709195029000913

In [ ]:
#KNNBaseline using pearson_baseline similarity only
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV


reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["pearson_baseline"],
    "min_support": [5],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8678481054944127
MAE: 0.6802883107211272
RMSE Best Params: {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': False}}
MAE Best Params: {'sim_options': {'name': 'pearson_baseline', 'min_support': 5, 'user_based': False}}


In [ ]:
# KNNBaseline using cosine similarity only
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV


reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [5],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.894247318239349
MAE: 0.7036931198323774
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}


In [ ]:
# KNNBaseline using cosine similarity only
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV


reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [3],
    "user_based": [False],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])


Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.895657047580421
MAE: 0.7049347872608015
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


#***KNNWithMeans*** 

Check by change the algorithm  and add n_jobs


In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [5],
    "user_based": [False],
    "algorithm": ["brute"],
    "n_jobs": [-1]
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)


print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])

alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8944551700185066
MAE: 0.7030400440184001
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
Computing the cosine similarity matrix...
Done computing similarity matrix.


#***KNNBasic***

Check by change the algorithm and add n_jobs

In [ ]:
from surprise import KNNBasic
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["cosine"],
    "min_support": [5],
    "user_based": [False],
    "algorithm": ["brute"],
    "n_jobs": [-1]
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)


print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])

alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9996317502098634
MAE: 0.7804705868491668
RMSE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
MAE Best Params: {'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBasic(name = 'cosine', min_support = 5, user_based = False, algorithm = 'brute', n_jobs = -1)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9258
MAE:  0.7303


0.7303039843973058

#***KNNBaseline***

Check by change the algorithm  and add n_jobs

In [ ]:
from surprise import KNNBaseline
from surprise import Dataset , Reader
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(df[['UserID','MovieID','Rating']],reader)
sim_options = {
    "name": ["pearson_baseline"],
    "min_support": [3],
    "user_based": [False],
    "algorithm": ["brute"],
    "n_jobs": [-1]
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)


print("RMSE:",gs.best_score["rmse"])
print("MAE:",gs.best_score["mae"])

print("RMSE Best Params:",gs.best_params["rmse"])
print("MAE Best Params:",gs.best_params["mae"])

alg = gs.best_estimator['rmse']
alg.fit(data.build_full_trainset())
print(alg)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8676366451223743
MAE: 0.6802497742555182
RMSE Best Params: {'sim_options': {'name': 'pearson_baseline', 'min_support': 3, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
MAE Best Params: {'sim_options': {'name': 'pearson_baseline', 'min_support': 3, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}}
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=.25)

algo=KNNBasic(name = 'pearson-baseline', min_support = 3, user_based = False, alrorithm = 'brute', n_jobs = -1)
predictions=algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9269
MAE:  0.7306


0.7306308538242435

# **Results**

***KNNWithMeans***

*   RMSE: 0.88773908475051
*   MAE: 0.6964841763563742

With best parameters{'name': 'msd', 'min_support': 5, 'user_based': False}

*After perform train test split on the best parametres*

*   RMSE: 0.9306
*   MAE:  0.7391

*KNNWithMeans* using cosine similarity 

*   RMSE: 0.89408056190716
*   MAE: 0.702771576599453


***KNNBasic***

*   RMSE: 0.9200638941257951
*   MAE: 0.723564229561002

With best parameters are{'name': 'msd', 'min_support': 5, 'user_based': False}

*After performing train test split on best parameters* 

*   RMSE: 0.9262
*   MAE:  0.7307

*KNNBasic* using cosine similarity

*   RMSE: 0.999738826435713
*   MAE: 0.7804487811659365

***KNNBaseline***

*   RMSE: 0.8876483738127501
*   MAE: 0.697069340378991

With best parameters are{'name': 'msd', 'min_support': 5, 'user_based': False}

*After performing train test split on best parameters*

*   RMSE: 0.8952
*   MAE:  0.7063

*Using pearson_baseline and cosine similarity to check which one is best*

*   RMSE: 0.8675298693325918
*   MAE: 0.6800854093886066

With pest parameters are{'name': 'pearson_baseline', 'min_support': 3, 'user_based': False}

*After performing train test split on above parameters*

*   RMSE: 0.8982
*   MAE:  0.7092

*KNNBaseline* using cosine similarity 

*   RMSE: 0.895657047580421
*   MAE: 0.7049347872608015

With parameters are{'name': 'cosine', 'min_support': 3, 'user_based': False}

**KNNWithMeans( change the algorithm and add n_jobs )**

*   RMSE: 0.8944551700185066
*   MAE: 0.7030400440184001

With parameters are {'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}

**KNNBasic( change the algorithm and add n_jobs )**

*   RMSE: 0.9996317502098634
*   MAE: 0.7804705868491668

With parameters are{'name': 'cosine', 'min_support': 5, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}

**KNNBaseline( change the algorithm and add n_jobs )**

*   RMSE: 0.8676366451223743
*   MAE: 0.6802497742555182

With parameters are{'name': 'pearson_baseline', 'min_support': 3, 'user_based': False, 'algorithm': 'brute', 'n_jobs': -1}

So, From all the above results the best RMSE and MAE given by KNNBaseline 
RMSE: 0.8675298693325918 ,
MAE: 0.6800854093886066
With best parameters are{'name': 'pearson_baseline', 'min_support': 3, 'user_based': False}